In [10]:
import os

# os.environ['CC'] = 'clang'
# os.environ['LDSHARED'] = 'clang -shared'

from timeit import timeit
import numpy as np

%reload_ext Cython
# ---------------------------------------------

# %%cython --annotate -c=-Ofast -c=-ffast-math -c=-mfpmath=sse -c=-funroll-loops -c=-march=native
# -c=-fopenmp -link-args=-fopenmp
# cython: boundscheck=False, wraparound=False, cdivision=True
# from libcpp cimport bool

In [23]:
%%cython --annotate -c=-Ofast -c=-ffast-math -c=-mfpmath=sse -c=-funroll-loops -c=-march=native -c=-fopenmp --link-args=-fopenmp
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

import numpy as np
from math import exp 
from libc.math cimport exp as c_exp

from cython.parallel import prange

def c_array_f_multi(double[:] X):

    cdef int N = X.shape[0]
    cdef double[:] Y = np.zeros(N)
    cdef int i

    # for i in prange(N, nogil=True):
    for i in range(N):
        if X[i] > 0.5:
            Y[i] = c_exp(X[i])
        else:
            Y[i] = 0

    return Y

In [24]:
import numpy as np

X = -1 + 2 * np.random.rand(10000000)

%timeit c_array_f_multi(X)

67.4 ms ± 804 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%cython --annotate -c=-Ofast -c=-ffast-math -c=-mfpmath=sse -c=-funroll-loops -c=-march=native -c=-fopenmp --link-args=-fopenmp
# cython: language_level=3, boundscheck=False, wraparound=False, cdivision=True

from cython.parallel import prange, parallel
import cython
# import numpy as np
cimport numpy as cnp

ctypedef cnp.npy_bool Bool
ctypedef long Int
ctypedef double Float
# ctypedef int Int
# ctypedef float Float

from libc.math cimport sqrt

# @cython.boundscheck(False)
# @cython.wraparound(False)
# @cython.cdivision(True)

cdef inline Int get_abs(Int x) nogil:
    if x > 0:
        return x
    return -x
 
cdef inline Int custom_hash(Int* cell, Int table_size) nogil:
    return get_abs((cell[0] * 92837111) ^ (cell[1] * 689287499) ^ (cell[2] * 283923481)) % table_size


cdef inline void cell_coord(Int* cell, Float* node, Float spacing_inv, Float move = 0) nogil:
    for i in range(3):
        cell[i] = <Int>((node[i] + move) * spacing_inv)

cdef inline Int cell_hash(Float* node, Float spacing_inv, Int table_size) nogil:
    cdef Int[3] cell
    cell_coord(cell, node, spacing_inv)
    return custom_hash(cell, table_size)



cdef inline Float get_min(Float* vector, int size) nogil:
    cdef Float result = vector[0]
    for i in range(1, size):
        if result > vector[i]:
            result = vector[i]
    return result


cdef inline void set_dot(Float* result, Float* matrix, Float* vector, int size) nogil:
    cdef Int i = 0, j = 0, k = 0
    for i in range(size):
         result[i] = 0.
         for j in range(size):
            k = i * size + j
            result[i] += (matrix[k] * vector[j])


cdef inline void set_diff(Float* result, Float* v1, Float* v2, int size) nogil:
    for i in range(size):
        result[i] = v1[i] - v2[i]


cdef Float get_det(Float* matrix) nogil:
    # cdef Float a11 = matrix[0], a12 = matrix[3], a13 = matrix[6]
    # cdef Float a21 = matrix[1], a22 = matrix[4], a23 = matrix[7]
    # cdef Float a31 = matrix[2], a32 = matrix[5], a33 = matrix[8]

    return matrix[0]* matrix[4]*matrix[8] + matrix[3]*matrix[7]*matrix[2] \
        + matrix[6]*matrix[1]*matrix[5] - matrix[6]*matrix[4]*matrix[2] \
        - matrix[3]*matrix[1]*matrix[8] - matrix[0]*matrix[7]*matrix[5]


# cdef Float get_det(Float[:,::1] matrix) nogil:
#     return matrix[0][0]*matrix[1][1]*matrix[2][2] + matrix[0][1]*matrix[1][2]*matrix[2][0] \
#         + matrix[0][2]*matrix[1][0]*matrix[2][1] - matrix[0][2]*matrix[1][1]*matrix[2][0] \
#         - matrix[0][1]*matrix[1][0]*matrix[2][2] - matrix[0][0]*matrix[1][2]*matrix[2][1]


cdef void set_matrix_inverse(Float* result, Float* matrix) nogil:
    cdef Float det = get_det(matrix)
    if (det == 0.0):
        # matrix[:] = 0.
        for i in range(9):
            matrix[i] = 0.
        return
    cdef Float det_inv = 1.0 / det
    
    result[0] =  (matrix[4] * matrix[8] - matrix[7] * matrix[5]) * det_inv
    result[3] = -(matrix[3] * matrix[8] - matrix[6] * matrix[5]) * det_inv
    result[6] =  (matrix[3] * matrix[7] - matrix[6] * matrix[4]) * det_inv
	
    result[1] = -(matrix[1] * matrix[8] - matrix[7] * matrix[2]) * det_inv
    result[4] =  (matrix[0] * matrix[8] - matrix[6] * matrix[2]) * det_inv
    result[7] = -(matrix[0] * matrix[7] - matrix[6] * matrix[1]) * det_inv
	
    result[2] =  (matrix[1] * matrix[5] - matrix[4] * matrix[2]) * det_inv
    result[5] = -(matrix[0] * matrix[5] - matrix[3] * matrix[2]) * det_inv
    result[8] =  (matrix[0] * matrix[4] - matrix[3] * matrix[1]) * det_inv


# cdef void set_matrix_inverse(Float* result, Float[:,::1] matrix) nogil:
#     cdef Float det = get_det(matrix)
#     if (det == 0.0):
#         matrix[:] = 0.
#         return
#     cdef Float det_inv = 1.0 / det

#     result[0] =  (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1]) * det_inv
#     result[1] = -(matrix[0][1] * matrix[2][2] - matrix[0][2] * matrix[2][1]) * det_inv
#     result[2] =  (matrix[0][1] * matrix[1][2] - matrix[0][2] * matrix[1][1]) * det_inv

#     result[3] = -(matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0]) * det_inv
#     result[4] =  (matrix[0][0] * matrix[2][2] - matrix[0][2] * matrix[2][0]) * det_inv
#     result[5] = -(matrix[0][0] * matrix[1][2] - matrix[0][2] * matrix[1][0]) * det_inv

#     result[6] =  (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix[2][0]) * det_inv
#     result[7] = -(matrix[0][0] * matrix[2][1] - matrix[0][1] * matrix[2][0]) * det_inv
#     result[8] =  (matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]) * det_inv



cdef void initialize_hasher(
    Float[:, ::1] nodes,
    Float spacing_inv,
    Int[::1] cell_starts,
    Int[::1] node_cell,
    Int table_size,
    Int nodes_count
) nogil:
    cell_starts[:] = 0
    cdef Float* node
    cdef int i = 0
    cdef int h
    for i in range(nodes_count):
        node = &nodes[i, 0]
        h = cell_hash(node=node, spacing_inv=spacing_inv, table_size=table_size)
        cell_starts[h] += 1

    cdef Int start = 0
    i = 0
    for i in range(table_size):
        start += cell_starts[i]
        cell_starts[i] = start
    cell_starts[table_size] = start  # guard

    i = 0
    for i in range(nodes_count):
        node = &nodes[i, 0]
        h = cell_hash(node=node, spacing_inv=spacing_inv, table_size=table_size)
        cell_starts[h] -= 1
        node_cell[cell_starts[h]] = i



cdef Int query_hasher(
    Int[::1] query_nodes,
    Bool[::1] ready_nodes_mask,
    Float* query_node,
    Float max_dist,
    Int[::1] cell_starts,
    Int[::1] node_cell,
    Int table_size,
    Float spacing_inv) nogil:

    cdef Int h, start, end, node_id
    cdef Int[3] cell_start, cell_stop, cell

    cell_coord(cell=&cell_start[0], node=&query_node[0], spacing_inv=spacing_inv, move = -max_dist)
    cell_coord(cell=&cell_stop[0], node=&query_node[0], spacing_inv=spacing_inv, move = +max_dist)

    cdef Int query_count = 0
    for c1 in range(cell_start[0], cell_stop[0]+1):
        for c2 in range(cell_start[1], cell_stop[1]+1):
            for c3 in range(cell_start[2], cell_stop[2]+1):
                cell[0] = c1
                cell[1] = c2
                cell[2] = c3

                h = custom_hash(cell=cell, table_size=table_size)
                start = cell_starts[h]
                end = cell_starts[h + 1]
                for i in range(start, end):
                    node_id = node_cell[i]
                    if not ready_nodes_mask[node_id]:
                        query_nodes[query_count] = node_id
                        query_count += 1
    return query_count


cdef Float get_norm(Float* vector, int size) nogil:
    cdef Float result = 0.
    for i in range(size):
        result += vector[i]**2
    return sqrt(result)

cdef void set_element_center(Float* element_center, Float* nodes, Int* element, int size) nogil:
    cdef Int node_start = 0, i = 0, j = 0
    for i in range(size):
        element_center[i] = 0.
    for j in range(4):
        node_start = size * element[j]
        for i in range(size):
            element_center[i] += nodes[node_start + i]
    for i in range(size):
        element_center[i] = element_center[i] / 4.


cdef void set_element_matrix_and_node(Float* element_matrix, Float* element_normalizing_node,
            Float* nodes, Int* element) nogil:
    cdef Int i = 0, n = 0
    for i in range(3):
        element_normalizing_node[i] = nodes[3 * element[3] + i]
    for n in range(3):
        for i in range(3):
            element_matrix[3 * i + n] = nodes[3 * element[n] + i] - element_normalizing_node[i]


cdef void complete_to_one(Float* weights, int size) nogil:
    weights[size] = 1.
    for i in range(size):
        weights[size] -= weights[i]

cdef Float get_element_radius(Float* element_center, Float* nodes, Int* element, int size) nogil:
    cdef Int j = 0
    cdef Float maximal_radius = 0, radius
    cdef Float[3] vector

    for j in range(4):
        node_start = size * element[j]
        set_diff(result=vector, v1=&element_center[0], v2=&nodes[node_start], size=3)
        radius = get_norm(vector=vector, size=3)
        if radius > maximal_radius:
            maximal_radius = radius

    return maximal_radius


cpdef void find_closest_nodes_cython(
    Int[:, ::1] closest_nodes,
    Float[:, ::1] closest_weights,
    Float[:, ::1] interpolated_nodes,
    Int[:, ::1] base_elements,
    Float[:, ::1] base_nodes,
    Int[::1] query_nodes,
    Bool[::1] ready_nodes_mask,
    Int[::1] cell_starts,
    Int[::1] node_cell,
    Float spacing,
    Float element_radius_padding
) nogil:
    cdef Int nodes_count = len(interpolated_nodes)
    cdef Int elements_count = len(base_elements)
    cdef Int table_size = len(cell_starts) - 1
    cdef Float spacing_inv = 1 / spacing

    initialize_hasher(nodes=interpolated_nodes, spacing_inv=spacing_inv, cell_starts=cell_starts, node_cell=node_cell, table_size=table_size, nodes_count=nodes_count)
    
    cdef Float[4] weights
    cdef Float[3] vector, element_center, element_normalizing_node
    cdef Float[9] element_matrix, element_matrix_inv
    for k in range(9):
        element_matrix[k] = 0.
        element_matrix_inv[k] = 0.
    cdef Float smallest_weight, smallest_current_weight, element_radius
    cdef Int node_id, i, query_id, query_count

    cdef Int element_id
    # with nogil, parallel():
    # for element_id in prange(elements_count, nogil=True): #num_threads, schedule
    for element_id in range(elements_count):
        # with gil:
        #     print(threadid())

        # make a function
        
        set_element_center(element_center=&element_center[0], nodes=&base_nodes[0][0], element=&base_elements[element_id][0], size=3)
        element_radius = get_element_radius(element_center=&element_center[0], nodes=&base_nodes[0][0], element=&base_elements[element_id][0], size=3)

        set_element_matrix_and_node(element_matrix, element_normalizing_node, nodes=&base_nodes[0][0], element=&base_elements[element_id][0])
        set_matrix_inverse(element_matrix_inv, element_matrix)
        # set_matrix_inverse(element_matrix_inv, &element_nodes_matrices_T[element_id][0][0])
        # set_matrix_inverse(matrix, element_nodes_matrices_T[element_id])

        query_count = query_hasher(
            query_nodes=query_nodes,
            ready_nodes_mask=ready_nodes_mask,
            query_node=element_center,
            max_dist=element_radius + element_radius_padding,
            cell_starts=cell_starts,
            node_cell=node_cell,
            table_size=table_size,
            spacing_inv=spacing_inv
        )

        # with parallel():
        # for query_id in prange(query_count):
        for query_id in range(query_count):
            node_id = query_nodes[query_id]

            # set_diff(result=vector, v1=&interpolated_nodes[node_id][0], v2=&normalizing_element_nodes_T[element_id][0], size=3)
            set_diff(result=vector, v1=&interpolated_nodes[node_id][0], v2=element_normalizing_node, size=3)
            set_dot(result=weights, matrix=element_matrix_inv, vector=vector, size=3)
                    
            # weights sum to one
            complete_to_one(weights=&weights[0], size=3)

            # looking for weights that are closest to positive
            smallest_weight = get_min(vector=&weights[0], size=4)
            smallest_current_weight = get_min(vector=&closest_weights[node_id][0], size=4)

            # better weight found
            if smallest_weight > smallest_current_weight:
                for i in range(4):
                    closest_weights[node_id, i] = weights[i]
                    closest_nodes[node_id, i] = base_elements[element_id][i]
            if smallest_weight >= 0:
                # positive weights found, only one element can contain node
               ready_nodes_mask[node_id] = True



In file included from /home/michal/anaconda3/envs/ml310/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948,
                 from /home/michal/anaconda3/envs/ml310/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/michal/anaconda3/envs/ml310/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/michal/.cache/ipython/cython/_cython_magic_0305decb4de01eb7003fa4b99323d44b.c:794:
/home/michal/anaconda3/envs/ml310/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [34]:
from time import time


def time_fun(fun):
    number = 1
    t = timeit(fun, number=number)
    print((t / number) * 1000, "ms")


def get_interlayer_data_skinning(
    base_nodes: np.ndarray,
    base_elements: np.ndarray,
    interpolated_nodes: np.ndarray,
    # padding: float,
):
    # padding = 0.0
    element_radius_padding = 0.01

    int_type = np.int64
    # int_type= np.int32
    # float_type = np.float32
    float_type = np.float64

    dim = base_nodes.shape[1]
    nodes_count = len(interpolated_nodes)
    elements_count = len(base_elements)
    closest_count = dim + 1
    element_nodes_count = base_elements.shape[1]

    spacing = 0.05
    table_size = 2 * elements_count

    table_size_proportion = 2
    table_size = table_size_proportion * nodes_count

    cell_starts = np.zeros(table_size + 1, dtype=int_type)
    node_cell = np.zeros(nodes_count, dtype=int_type)

    closest_distances = np.zeros((nodes_count, closest_count))
    closest_nodes = np.full_like(closest_distances, fill_value=-1, dtype=int_type)
    closest_weights = np.full_like(closest_distances, fill_value=-1e8, dtype=float_type)

    ready_nodes_mask = np.zeros(nodes_count, dtype=np.bool_)
    nodes_query = np.zeros(nodes_count, dtype=int_type)
    element_nodes = np.zeros((elements_count, 4, 3))

    t = time()
    find_closest_nodes_cython(
        closest_nodes=closest_nodes,
        closest_weights=closest_weights,
        interpolated_nodes=interpolated_nodes.astype(float_type),
        base_elements=base_elements.astype(int_type),
        base_nodes=base_nodes,
        query_nodes=nodes_query,
        ready_nodes_mask=ready_nodes_mask,
        cell_starts=cell_starts,
        node_cell=node_cell,
        spacing=spacing,
        element_radius_padding=element_radius_padding,
    )
    stop = time() - t

    # assert np.all(closest_nodes >= 0)  # For each node at least one element found
    print("Min weight", closest_weights.min())
    print("Time all ms: ", 1000 * stop)
    print("Ready ", ready_nodes_mask.sum())
    print("nodes ", nodes_count)
    return closest_nodes, closest_distances, closest_weights


""
base_nodes = np.load("../output/base_nodes.npy")
base_elements = np.load("../output/base_elements.npy")
interpolated_nodes = np.load("../output/interpolated_nodes.npy")

(
    closest_nodes,
    closest_distances,
    closest_weights,
) = get_interlayer_data_skinning(  # time_fun(lambda :
    base_nodes,
    base_elements,
    interpolated_nodes,
    # padding: float,
)
# for i in range(10):
#     closest_nodes, closest_distances, closest_weights = get_interlayer_data_skinning( #time_fun(lambda :
#         base_nodes[:13_000],
#         base_elements[:13_000],
#         interpolated_nodes[:13_000],
#         # padding: float,
#     )

# Time all ms:  3136.69490814209
# Time all ms:  2542.71 # 249?

Min weight 1.9740281483409605e-07
Time all ms:  2652.0674228668213
Ready  322881
nodes  322881


In [29]:
closest_nodes.sum()

32266731251

In [26]:
closest_weights

array([[0.38243691, 0.03486463, 0.27255838, 0.31014008],
       [0.29784048, 0.08666121, 0.41717339, 0.19832492],
       [0.11332874, 0.14836075, 0.11738075, 0.62092976],
       ...,
       [0.70778347, 0.08884906, 0.13689966, 0.06646781],
       [0.36366614, 0.04821793, 0.45232448, 0.13579145],
       [0.13084401, 0.14246696, 0.00508829, 0.72160075]])

In [30]:
A = np.array(
    [
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2],
    ],
    dtype=np.float64,
)
# set_matrix_inverse(A)
# A

vector = np.array([1.0, 2.0, 3.0])
result = np.array([1.0, 1.0, 1.0])
set_dot(result, A, vector)

result

NameError: name 'set_dot' is not defined

In [ ]:
nodes_count = 43_000  # 43_000 #14_000
elements_count = 30_000  # 30_000 #14_000

int_type = np.int64  # 64)
float_type = np.float64

radius = 0.025
spacing = 2 * radius  # 3
max_dist = 0.0
table_size = 2 * elements_count
spacing_inv = 1 / spacing


interpolated_nodes = np.random.rand(nodes_count, 3).astype(float_type)
element_centers = np.random.rand(elements_count, 3).astype(float_type)
element_ball_radiuses = np.zeros(elements_count, dtype=float_type)
element_ball_radiuses[:] = radius
query_nodes = np.zeros(elements_count, dtype=int_type)
ready_nodes_mask = np.zeros(nodes_count, dtype=np.bool_)

cell_starts = np.zeros(table_size, dtype=int_type)
node_cell = np.zeros(nodes_count, dtype=int_type)


# node_cell = np.zeros(nodes_count, dtype=np.int64)
# time(lambda : initialize_hasher(nodes=interpolated_nodes, spacing_inv=spacing_inv, cell_starts=cell_starts, node_cell=node_cell))

# nodes_mask = np.zeros(nodes_count, dtype=np.bool_)
# time(lambda : query_hasher(nodes_mask=nodes_mask, query_node=interpolated_nodes[0], max_dist=max_dist, cell_starts=cell_starts, node_cell=node_cell, spacing_inv=spacing_inv))

time_fun(
    lambda: find_closest_nodes_cython(
        interpolated_nodes=interpolated_nodes,
        element_centers=element_centers,
        element_ball_radiuses=element_ball_radiuses,
        query_nodes=query_nodes,
        query_node=interpolated_nodes[0],
        ready_nodes_mask=ready_nodes_mask,
        max_dist=max_dist,
        cell_starts=cell_starts,
        node_cell=node_cell,
        spacing_inv=spacing_inv,
    )
)

TypeError: find_closest_nodes_cython() takes exactly 14 positional arguments (0 given)

In [ ]:
nodes_mask.sum()

10291